In [ ]:
!pip install -q kaggle ultralytics xmltodict


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.5/974.5 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 28.4 MB/s eta 0:00:00


In [ ]:
import os
import json

# Set up Kaggle API credentials
os.makedirs('/root/.kaggle', exist_ok=True)

kaggle_json = {
    "username": "jun4id",  # 🔁 Replace with your actual Kaggle username if different
    "key": "727ddfb29983b0f606feccd167776920"  # 🔁 Replace with your Kaggle API key if different
}

with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump(kaggle_json, f)

# Set proper permissions for the Kaggle file
!chmod 600 /root/.kaggle/kaggle.json


In [ ]:
# Download datasets from Kaggle
!kaggle datasets download -d hakantaskiner/personforklift-dataset
!kaggle datasets download -d saharshsinha/forklift-dataset


Dataset URL: https://www.kaggle.com/datasets/hakantaskiner/personforklift-dataset
License(s): CC0-1.0
Dataset URL: https://www.kaggle.com/datasets/saharshsinha/forklift-dataset
License(s): unknown


In [ ]:
import zipfile
import os
import time

# Create data directory
os.makedirs('/content/data', exist_ok=True)

# Extract personforklift dataset
with zipfile.ZipFile('personforklift-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/data')

# Extract forklift dataset
with zipfile.ZipFile('forklift-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/data')

# Wait a bit to ensure extraction completes
time.sleep(2)

# Show extracted directory contents
!ls -la /content/data


total 16
drwxr-xr-x 4 root root 4096 Apr 12 14:36 .
drwxr-xr-x 1 root root 4096 Apr 12 14:36 ..
drwxr-xr-x 4 root root 4096 Apr 12 14:36 hf-dataset
drwxr-xr-x 5 root root 4096 Apr 12 14:36 images-


In [ ]:
# Define base directory for YOLO dataset
base_dir = '/content/forklift_dataset'

# Create train, val, test folders with subfolders for images and labels
for split in ['train', 'val', 'test']:
    os.makedirs(f'{base_dir}/{split}/images', exist_ok=True)
    os.makedirs(f'{base_dir}/{split}/labels', exist_ok=True)

print("✅ Created YOLOv8 dataset folder structure.")


✅ Created YOLOv8 dataset folder structure.


In [ ]:
import xmltodict
import os

def convert_xml_to_yolo(xml_file, output_dir):
    """Convert XML annotation file to YOLO format"""
    try:
        # Parse XML file
        with open(xml_file, 'r') as f:
            xml_content = f.read()
        xml_dict = xmltodict.parse(xml_content)

        # Get image dimensions
        try:
            width = int(xml_dict['annotation']['size']['width'])
            height = int(xml_dict['annotation']['size']['height'])
        except:
            print(f"Error: Missing size info in {xml_file}")
            return None

        # Extract objects
        yolo_lines = []
        objects = xml_dict['annotation'].get('object', [])

        # Handle single object case
        if not isinstance(objects, list):
            objects = [objects]

        # Process each object
        for obj in objects:
            try:
                class_name = obj['name'].lower()
                if 'forklift' not in class_name:
                    continue

                box = obj['bndbox']
                xmin = float(box['xmin'])
                ymin = float(box['ymin'])
                xmax = float(box['xmax'])
                ymax = float(box['ymax'])

                x_center = ((xmin + xmax) / 2) / width
                y_center = ((ymin + ymax) / 2) / height
                box_width = (xmax - xmin) / width
                box_height = (ymax - ymin) / height

                yolo_lines.append(f"0 {x_center} {y_center} {box_width} {box_height}")
            except Exception as e:
                print(f"Error in {xml_file}: {e}")
                continue

        if not yolo_lines:
            return None

        output_file = os.path.join(output_dir, os.path.basename(xml_file).replace('.xml', '.txt'))
        with open(output_file, 'w') as f:
            f.write('\n'.join(yolo_lines))

        return output_file
    except Exception as e:
        print(f"Failed to process {xml_file}: {e}")
        return None


In [ ]:
import glob
import shutil
import random

def process_dataset(images_dir, annotations_dir, split_ratios=[0.7, 0.2, 0.1]):
    """Process dataset, convert annotations, and split into train/val/test"""

    image_files = []
    for ext in ['.jpg', '.jpeg', '.png']:
        image_files.extend(glob.glob(os.path.join(images_dir, f'*{ext}')))

    if not image_files:
        print(f"No images found in {images_dir}")
        return

    print(f"Found {len(image_files)} images in {images_dir}")
    random.shuffle(image_files)

    train_end = int(len(image_files) * split_ratios[0])
    val_end = train_end + int(len(image_files) * split_ratios[1])

    train_images = image_files[:train_end]
    val_images = image_files[train_end:val_end]
    test_images = image_files[val_end:]

    splits = {
        'train': train_images,
        'val': val_images,
        'test': test_images
    }

    for split_name, split_images in splits.items():
        print(f"Processing {len(split_images)} images for {split_name} split...")
        for img_path in split_images:
            try:
                img_filename = os.path.basename(img_path)
                base_filename = os.path.splitext(img_filename)[0]
                xml_filename = base_filename + '.xml'

                dst_img_path = os.path.join(base_dir, split_name, 'images', img_filename)
                shutil.copy(img_path, dst_img_path)

                xml_path = os.path.join(annotations_dir, xml_filename)
                if os.path.exists(xml_path):
                    labels_dir = os.path.join(base_dir, split_name, 'labels')
                    convert_xml_to_yolo(xml_path, labels_dir)
                else:
                    print(f"Annotation not found for {img_filename}")
            except Exception as e:
                print(f"Error processing {img_path}: {e}")


In [ ]:
print("\nAnalyzing dataset structure...")

# Check first dataset structure (hf-dataset)
if os.path.exists('/content/data/hf-dataset'):
    print("Found hf-dataset")
    hf_images_dir = '/content/data/hf-dataset/images'
    hf_labels_dir = '/content/data/hf-dataset/labels'

    if os.path.exists(os.path.join(hf_images_dir, 'train')):
        hf_images_dir = os.path.join(hf_images_dir, 'train')
        print(f"Images found in train subdirectory: {hf_images_dir}")
else:
    hf_dirs = glob.glob('/content/data/**/hf-dataset', recursive=True)
    if hf_dirs:
        print(f"Found hf-dataset at {hf_dirs[0]}")
        hf_images_dir = os.path.join(hf_dirs[0], 'images')
        hf_labels_dir = os.path.join(hf_dirs[0], 'labels')

        if os.path.exists(os.path.join(hf_images_dir, 'train')):
            hf_images_dir = os.path.join(hf_images_dir, 'train')
            print(f"Images found in train subdirectory: {hf_images_dir}")
    else:
        print("Could not find hf-dataset")
        hf_images_dir = None
        hf_labels_dir = None

# Check second dataset structure (images-)
if os.path.exists('/content/data/images-'):
    print("Found images- dataset")
    images_dir = '/content/data/images-/train'
else:
    images_dirs = glob.glob('/content/data/**/images-', recursive=True)
    if images_dirs:
        print(f"Found images- dataset at {images_dirs[0]}")
        images_dir = os.path.join(images_dirs[0], 'train')
    else:
        print("Could not find images- dataset")
        images_dir = None

# Process both datasets
if hf_images_dir and hf_labels_dir and os.path.exists(hf_images_dir) and os.path.exists(hf_labels_dir):
    print(f"\nProcessing first dataset (HF dataset) from {hf_images_dir}...")
    process_dataset(hf_images_dir, hf_labels_dir)
else:
    print("\nSkipping first dataset (HF dataset) as it was not found or is incomplete")

if images_dir and os.path.exists(images_dir):
    print(f"\nProcessing second dataset (images-) from {images_dir}...")
    process_dataset(images_dir, images_dir)
else:
    print("\nSkipping second dataset (images-) as it was not found")



Analyzing dataset structure...
Found hf-dataset
Images found in train subdirectory: /content/data/hf-dataset/images/train
Found images- dataset

Processing first dataset (HF dataset) from /content/data/hf-dataset/images/train...
Found 1249 images in /content/data/hf-dataset/images/train
Processing 874 images for train split...
Annotation not found for data_261.jpg
Annotation not found for SQ1SU8KS5WNU_jpg.rf.1490e0727e20d7c0d5cbf62276817665.jpg
Annotation not found for datax_132.jpg
Annotation not found for dataq_75.jpg
Annotation not found for dataq_114.jpg
Annotation not found for image_341.jpg
Annotation not found for data_10.jpg
Annotation not found for HILZJIWVPORL_jpg.rf.26acc4deb2f0c0667ca4a09ca046c390.jpg
Annotation not found for X7LY4HTFT2XE_jpg.rf.b171bec13657347f0bd0af194cd38960.jpg
Annotation not found for NVL8YR7VSQ04_jpg.rf.5d32a227590c79db13af3f53bb45f5ae.jpg
Annotation not found for 62LXGO7FCDAQ_jpg.rf.466228aac3eae9cdb9408bd443c444c4.jpg
Annotation not found for dataq

In [ ]:
# Sanity check processed folders
for split in ['train', 'val', 'test']:
    img_path = f"/content/dataset/images/{split}"
    lbl_path = f"/content/dataset/labels/{split}"
    print(f"\n{split.upper()} SET:")
    print(f" - Images: {len(glob.glob(img_path + '/*.jpg'))}")
    print(f" - Labels: {len(glob.glob(lbl_path + '/*.txt'))}")



TRAIN SET:
 - Images: 0
 - Labels: 0

VAL SET:
 - Images: 0
 - Labels: 0

TEST SET:
 - Images: 0
 - Labels: 0


In [ ]:
import shutil
import os

def move_files(src_img_dir, src_lbl_dir, dst_img_dir, dst_lbl_dir):
    os.makedirs(dst_img_dir, exist_ok=True)
    os.makedirs(dst_lbl_dir, exist_ok=True)

    for img_file in glob.glob(f"{src_img_dir}/*.jpg"):
        shutil.copy(img_file, dst_img_dir)

    for lbl_file in glob.glob(f"{src_lbl_dir}/*.txt"):
        shutil.copy(lbl_file, dst_lbl_dir)

# Move HF dataset
move_files('/content/data/hf-dataset/images/train', '/content/data/hf-dataset/labels/train',
           '/content/dataset/images/train', '/content/dataset/labels/train')

# Move images- dataset (train, val, test)
for split in ['train', 'val', 'test']:
    move_files(f'/content/data/images-/images/{split}', f'/content/data/images-/labels/{split}',
               f'/content/dataset/images/{split}', f'/content/dataset/labels/{split}')


In [ ]:
import os

for split in ['train', 'val', 'test']:
    img_path = f'/content/dataset/images/{split}'
    lbl_path = f'/content/dataset/labels/{split}'

    img_count = len(os.listdir(img_path)) if os.path.exists(img_path) else 0
    lbl_count = len(os.listdir(lbl_path)) if os.path.exists(lbl_path) else 0

    print(f"{split.upper()} SET:")
    print(f" - Images: {img_count}")
    print(f" - Labels: {lbl_count}")
    print()


TRAIN SET:
 - Images: 1249
 - Labels: 1249

VAL SET:
 - Images: 0
 - Labels: 0

TEST SET:
 - Images: 0
 - Labels: 0



In [ ]:
import os
import random
import shutil
from pathlib import Path

# Paths
image_dir = Path('/content/dataset/images/train')
label_dir = Path('/content/dataset/labels/train')

val_img_dir = Path('/content/dataset/images/val')
val_lbl_dir = Path('/content/dataset/labels/val')

test_img_dir = Path('/content/dataset/images/test')
test_lbl_dir = Path('/content/dataset/labels/test')

# Create val and test directories
for d in [val_img_dir, val_lbl_dir, test_img_dir, test_lbl_dir]:
    d.mkdir(parents=True, exist_ok=True)

# List all image files
image_files = list(image_dir.glob('*.jpg'))

# Shuffle for randomness
random.shuffle(image_files)

# Define split counts
val_count = int(0.1 * len(image_files))
test_count = int(0.1 * len(image_files))

val_files = image_files[:val_count]
test_files = image_files[val_count:val_count + test_count]

# Function to move files
def move_files(file_list, target_img_dir, target_lbl_dir):
    for img_path in file_list:
        lbl_path = label_dir / f"{img_path.stem}.txt"
        if lbl_path.exists():
            shutil.move(str(img_path), str(target_img_dir / img_path.name))
            shutil.move(str(lbl_path), str(target_lbl_dir / lbl_path.name))

# Move validation and test files
move_files(val_files, val_img_dir, val_lbl_dir)
move_files(test_files, test_img_dir, test_lbl_dir)

print(f"Moved {len(val_files)} images to val and {len(test_files)} images to test.")


Moved 124 images to val and 124 images to test.


In [ ]:
from pathlib import Path

def count_files(path):
    images = list((Path(path) / 'images').glob('**/*.jpg'))
    labels = list((Path(path) / 'labels').glob('**/*.txt'))
    return len(images), len(labels)

splits = ['train', 'val', 'test']
for split in splits:
    img_count, lbl_count = count_files(f'/content/dataset/{split}')
    print(f"{split.upper()} SET:\n - Images: {img_count}\n - Labels: {lbl_count}\n")


TRAIN SET:
 - Images: 0
 - Labels: 0

VAL SET:
 - Images: 0
 - Labels: 0

TEST SET:
 - Images: 0
 - Labels: 0



In [ ]:
import shutil
from pathlib import Path

source_base = Path('/content/data/hf-dataset-split')
target_base = Path('/content/dataset')

for split in ['train', 'val', 'test']:
    for kind in ['images', 'labels']:
        src = source_base / split / kind
        dst = target_base / split / kind
        dst.mkdir(parents=True, exist_ok=True)
        for file in src.glob('*'):
            shutil.copy(file, dst)

print("✅ Files copied to YOLO dataset directory.")


✅ Files copied to YOLO dataset directory.


In [ ]:
from pathlib import Path

dataset_path = Path('/content/dataset')

for split in ['train', 'val', 'test']:
    img_count = len(list((dataset_path / split / 'images').glob('*')))
    lbl_count = len(list((dataset_path / split / 'labels').glob('*')))
    print(f"{split.upper()} SET:\n - Images: {img_count}\n - Labels: {lbl_count}\n")


TRAIN SET:
 - Images: 0
 - Labels: 0

VAL SET:
 - Images: 0
 - Labels: 0

TEST SET:
 - Images: 0
 - Labels: 0



In [ ]:
!ls -l /content/dataset/train/images | head -20
!ls -l /content/dataset/train/labels | head -20
!ls -l /content/dataset/val/images | head -20
!ls -l /content/dataset/val/labels | head -20
!ls -l /content/dataset/test/images | head -20
!ls -l /content/dataset/test/labels | head -20


total 0
total 0
total 0
total 0
total 0
total 0


In [ ]:
!ls -R /content/split_dataset


ls: cannot access '/content/split_dataset': No such file or directory


In [ ]:
import os
import random
import shutil
from pathlib import Path

# Set random seed for reproducibility
random.seed(42)

# Paths
dataset_dir = Path('/content/data/hf-dataset/images/train')
label_dir = Path('/content/data/hf-dataset/labels/train')
output_dir = Path('/content/split_dataset')

# Ensure output structure
for split in ['train', 'val', 'test']:
    (output_dir / split / 'images').mkdir(parents=True, exist_ok=True)
    (output_dir / split / 'labels').mkdir(parents=True, exist_ok=True)

# Load all image filenames (excluding those without annotations)
image_paths = sorted(dataset_dir.glob('*.jpg'))
valid_image_paths = [img for img in image_paths if (label_dir / (img.stem + '.txt')).exists()]

# Split: 80% train, 10% val, 10% test
n_total = len(valid_image_paths)
n_train = int(n_total * 0.8)
n_val = int(n_total * 0.1)
n_test = n_total - n_train - n_val

train_imgs = valid_image_paths[:n_train]
val_imgs = valid_image_paths[n_train:n_train + n_val]
test_imgs = valid_image_paths[n_train + n_val:]

# Function to move image + label
def move_files(img_paths, split):
    for img_path in img_paths:
        label_path = label_dir / (img_path.stem + '.txt')
        dst_img = output_dir / split / 'images' / img_path.name
        dst_lbl = output_dir / split / 'labels' / label_path.name
        shutil.copy(img_path, dst_img)
        shutil.copy(label_path, dst_lbl)

move_files(train_imgs, 'train')
move_files(val_imgs, 'val')
move_files(test_imgs, 'test')

print("✅ Done splitting dataset:")
print(f"Train: {len(train_imgs)} images")
print(f"Val:   {len(val_imgs)} images")
print(f"Test:  {len(test_imgs)} images")


✅ Done splitting dataset:
Train: 999 images
Val:   124 images
Test:  126 images


In [ ]:
import shutil
from pathlib import Path

# Define source and destination base paths
src_base = Path('/content/split_dataset')
dst_base = Path('/content/yolo_dataset')

# Copy each split (images and labels)
for split in ['train', 'val', 'test']:
    for subfolder in ['images', 'labels']:
        src = src_base / split / subfolder
        dst = dst_base / split / subfolder

        if dst.exists():
            shutil.rmtree(dst)  # remove old contents if any

        shutil.copytree(src, dst)
        print(f"✅ Copied {split}/{subfolder} from {src} to {dst}")


✅ Copied train/images from /content/split_dataset/train/images to /content/yolo_dataset/train/images
✅ Copied train/labels from /content/split_dataset/train/labels to /content/yolo_dataset/train/labels
✅ Copied val/images from /content/split_dataset/val/images to /content/yolo_dataset/val/images
✅ Copied val/labels from /content/split_dataset/val/labels to /content/yolo_dataset/val/labels
✅ Copied test/images from /content/split_dataset/test/images to /content/yolo_dataset/test/images
✅ Copied test/labels from /content/split_dataset/test/labels to /content/yolo_dataset/test/labels


In [ ]:
data_yaml = """
path: /content/yolo_dataset
train: train/images
val: val/images
test: test/images

names:
  0: forklift
  1: person
"""

with open('/content/yolo_dataset/data.yaml', 'w') as f:
    f.write(data_yaml.strip())

print("✅ Created data.yaml for YOLOv8")


✅ Created data.yaml for YOLOv8


In [ ]:
from ultralytics import YOLO

# Load a YOLOv8 model (you can change 'yolov8n.pt' to yolov8s.pt, yolov8m.pt, etc. for larger models)
model = YOLO('yolov8n.pt')  # or 'yolov8s.pt'

# Train the model
model.train(
    data='/content/yolo_dataset/data.yaml',
    epochs=100,
    imgsz=640,
    batch=16,
    name='forklift_detector'
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 103MB/s]


Ultralytics 8.3.107 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/yolo_dataset/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=forklift_detector, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes

100%|██████████| 755k/755k [00:00<00:00, 21.6MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

                   all        124        136      0.575      0.706      0.648      0.463



100 epochs completed in 0.548 hours.
Optimizer stripped from runs/detect/forklift_detector/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/forklift_detector/weights/best.pt, 6.2MB

Validating runs/detect/forklift_detector/weights/best.pt...
Ultralytics 8.3.107 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.66it/s]


                   all        124        136      0.673      0.729      0.709       0.51
              forklift        123        125      0.974       0.64      0.902      0.614
                person         11         11      0.373      0.818      0.517      0.405
Speed: 0.4ms preprocess, 2.1ms inference, 0.0ms loss, 5.2ms postprocess per image
Results saved to runs/detect/forklift_detector


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ea20a5c9dd0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [ ]:
!find /content/runs -name "best.pt"


/content/runs/detect/forklift_detector/weights/best.pt


In [ ]:
from ultralytics import YOLO

# Load your trained model
model = YOLO('/content/runs/detect/forklift_detector/weights/best.pt')

# Run inference on the new video
results = model.predict(source='/content/forkliftvideo.mp4', save=True, conf=0.25)




WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1522) /content/forkliftvideo.mp4: 384x640 (no detections), 17.8ms
video 1/1 (frame 2/1522) /content/forkliftvideo.mp4: 384x640 (no detections), 12.5ms
video 1/1 (frame 3/1522) /content/forkliftvideo.mp4: 384x640 (no detections), 12.0ms
video 1/1 (frame 4/1522) /content/forkliftvideo.mp4: 384x640 (no detections), 13.8ms
video 1/1 (frame 5/1522) /content/forkliftvideo.mp4: 384x640 (no detections), 12.1ms
video 1/1 (frame 6/1522) /conte

In [ ]:
!zip -r yolo_dataset.zip /content/yolo_dataset


  adding: content/yolo_dataset/ (stored 0%)
  adding: content/yolo_dataset/val/ (stored 0%)
  adding: content/yolo_dataset/val/labels.cache (deflated 86%)
  adding: content/yolo_dataset/val/labels/ (stored 0%)
  adding: content/yolo_dataset/val/labels/image_248.txt (deflated 15%)
  adding: content/yolo_dataset/val/labels/image_279.txt (deflated 23%)
  adding: content/yolo_dataset/val/labels/image_226.txt (deflated 5%)
  adding: content/yolo_dataset/val/labels/image_229.txt (deflated 5%)
  adding: content/yolo_dataset/val/labels/image_206.txt (deflated 10%)
  adding: content/yolo_dataset/val/labels/image_293.txt (deflated 21%)
  adding: content/yolo_dataset/val/labels/image_290.txt (deflated 13%)
  adding: content/yolo_dataset/val/labels/image_233.txt (deflated 18%)
  adding: content/yolo_dataset/val/labels/image_270.txt (deflated 8%)
  adding: content/yolo_dataset/val/labels/image_240.txt (deflated 8%)
  adding: content/yolo_dataset/val/labels/image_227.txt (deflated 23%)
  adding: con

In [ ]:
from google.colab import files
files.download("yolo_dataset.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download("/content/yolo_dataset/data.yaml")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>